In [1]:
import pymc3 as pm
import numpy as np
import scipy.stats as stats
import scipy.special as special
import matplotlib.pyplot as plt
import arviz as az
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf 

from sklearn import preprocessing
from theano import shared

In [2]:
%config InlineBackend.figure_format = 'retina'
az.style.use('arviz-darkgrid')

In [3]:
import sys, IPython, scipy, matplotlib, pandas, seaborn, patsy, platform, theano, sklearn, statsmodels
print("""This notebook was created using:
Python {}
IPython {}
PyMC3 {}
ArviZ {}
NumPy {}
SciPy {}
Pandas {}
Seaborn {}
Patsy {}
Matplotlib {}
Theano {}
Sklearn {}
Statsmodels {}\n""".format(sys.version[:5], 
                             IPython.__version__, 
                             pm.__version__, 
                             az.__version__, 
                             np.__version__, 
                             scipy.__version__, 
                             pandas.__version__, 
                             seaborn.__version__, 
                             patsy.__version__, 
                             matplotlib.__version__, 
                             theano.__version__, 
                             sklearn.__version__, 
                             statsmodels.api.__version__))

This notebook was created using:
Python 3.7.4
IPython 7.13.0
PyMC3 3.8
ArviZ 0.7.0
NumPy 1.18.2
SciPy 1.4.1
Pandas 1.0.3
Seaborn 0.10.0
Patsy 0.5.1
Matplotlib 3.2.1
Theano 1.0.4
Sklearn 0.22.2.post1
Statsmodels 0.11.1



### 1. Consider three fictional Polynesian islands. On each there is a Royal Ornithologist charged by the king with surveying the birb population. They have each found the following proportions of 5 important birb species:

In [4]:
# compose the data frame
df = pd.DataFrame({'Birb_A': [0.2, 0.8, 0.05],
                   'Birb_B': [0.2, 0.1, 0.15],
                   'Birb_C': [0.2, 0.05, 0.7],
                   'Birb_D': [0.2, 0.025, 0.05],
                   'Birb_E': [0.2, 0.025, 0.05]})
df.head()

,Birb_A,Birb_B,Birb_C,Birb_D,Birb_E
0,0.20,0.20,0.20,0.200,0.200
1,0.80,0.10,0.05,0.025,0.025
2,0.05,0.15,0.70,0.050,0.050


### Notice that each row sums to 1, all the birbs. This problem has two parts. It is not computationally complicated. But it is conceptually tricky.

### First, compute the entropy of each island’s birb distribution. Interpret these entropy values.

### Second, use each island’s birb distribution to predict the other two. This means to compute the K-L Divergence of each island from the others, treating each island as if it were a statistical model of the other islands. You should end up with 6 different K-L Divergence values. Which island predicts the others best? Why?

In [5]:
entropy = np.zeros(len(df))
for i in range(len(df)):
    entropy[i] = -sum(np.log(df.iloc[i].values)*df.iloc[i].values)
entropy

array([1.60943791, 0.74300394, 0.9836003 ])

The first island has the largest entropy, followed by the third, and then the second in last place. Why is this? Entropy is a measure of the evenness of a distribution. The first islands has the most even distribution of birbs. This means you wouldn’t be very surprised by any particular birb. The second island, in contrast, has a very uneven distribution of birbs. If you saw any birb other than the first species, it would be surprising.

K-L divergence is the average difference in $log$ probability between the target ($p$) and model ($q$). 

$D_{KL}(p,q) = \sum_{i=1}p_i(log(p_i)-log(q_i))$

In [6]:
D_KL = np.zeros([len(df), len(df)])

In [7]:
def DKL(p,q):
    return sum(p*(np.log(p)-np.log(q)))

In [8]:
for i in range(len(df)):
    for j in range(len(df)):
        if i!=j:
            D_KL[i,j] = DKL(df.iloc[j].values, df.iloc[i].values)  
D_KL

array([[0.        , 0.86643398, 0.62583761],
       [0.97040605, 0.        , 1.83884518],
       [0.63876044, 2.01091424, 0.        ]])

The way to read this is each row as a model and each column as a true dis- tribution. So the first island, the first row, has the smaller distances to the other islands. This makes sense, since it has the highest entropy. Why does that give it a shorter distance to the other islands? Because it is less surprised by the other islands, due to its high entropy.

### 2. Recall the marriage, age, and happiness collider bias example from Chapter 6. Run models m6.9 and m6.10 again. Compare these two models using WAIC (or LOO, they will produce identical results). Which model is expected to make better predictions? Which model provides the correct causal inference about the influence of age on happiness? Can you explain why the answers to these two questions disagree?

In [9]:
def sim_happiness(N_years=1000, max_age=65, N_births=20, aom=18):
    H=[0]
    M=[0]
    A=[0]
    for t in range(N_years):
        # age existing individuals
        A = [x + 1 for x in A]
        # newborns
        A = A + np.repeat(1, N_births).tolist()
        # sim happiness trait - never changes
        H = H + np.linspace(-2, 2, N_births).tolist()
        # not yet married
        M = M + np.repeat(0, N_births).tolist()
        # for each person over 17, chance get married
        for i in range(len(A)):
            if A[i]>=aom and M[i]==0:
                M[i] = int(np.random.binomial(n=1, p=special.expit(H[i]-4), size=1))
        # mortality
        deaths = []
        for i in range(len(A)):
            if A[i]>max_age:
                deaths.append(i)
        A = [x for i,x in enumerate(A) if i not in deaths]
        H = [x for i,x in enumerate(H) if i not in deaths]
        M = [x for i,x in enumerate(M) if i not in deaths]
        #print(A)
        #print(M)
    # compose a clean data frame
    d = pd.DataFrame({'age': A,
                      'married': M,
                      'happiness':H})
    return d

In [10]:
df = sim_happiness()
df.describe()

,age,married,happiness
count,1300.000000,1300.000000,1.300000e+03
mean,33.000000,0.286154,-8.335213e-17
std,18.768883,0.452136,1.214421e+00
min,1.000000,0.000000,-2.000000e+00
25%,17.000000,0.000000,-1.000000e+00
50%,33.000000,0.000000,-1.110223e-16
75%,49.000000,1.000000,1.000000e+00
max,65.000000,1.000000,2.000000e+00


In [11]:
# filter out people < 17
df2 = df[df['age']>17]
df2['A'] = (df2['age']-18)/(65-18)
df2.head()

/Users/yifei-wang/.pyenv/versions/3.7.4/envs/Statistical_Rethinking/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,age,married,happiness,A
0,65,0,-2.000000,1.0
1,65,1,-1.789474,1.0
2,65,0,-1.578947,1.0
3,65,0,-1.368421,1.0
4,65,0,-1.157895,1.0


In [12]:
mid = df2['married'].values
# define model: influence of age on happiness, while controlling for marriage status.
# μ_i = α_mid[i] + β_A*A_i
with pm.Model() as m6_9:
    a = pm.Normal('a', mu=0, sd=2, shape=2)
    bA = pm.Normal('bA', mu=0, sd=2)
    sigma = pm.Exponential('sigma', 1)
    mu = pm.Deterministic('mu', a[mid] + bA*df2['A'])
    happiness = pm.Normal('happiness', mu=mu, sd=sigma, observed=df2['happiness'])
    m6_9_trace = pm.sample(1000, tune=1000)

# show model summary
varnames = ['~mu']
az.summary(m6_9_trace, varnames, kind="stats", round_to=2, credible_interval=0.89)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, bA, a]
Sampling 4 chains, 0 divergences: 100%|██████████| 8000/8000 [00:02<00:00, 2845.65draws/s]


,mean,sd,hpd_5.5%,hpd_94.5%
a[0],-0.22,0.06,-0.32,-0.12
a[1],1.27,0.09,1.13,1.41
bA,-0.72,0.11,-0.89,-0.53
sigma,1.00,0.02,0.96,1.03


In [13]:
# define model: influence of age on happiness without controlling for marriage status.
# μ_i = α + β_A*A_i
with pm.Model() as m6_10:
    a = pm.Normal('a', mu=0, sd=1)
    bA = pm.Normal('bA', mu=0, sd=2)
    sigma = pm.Exponential('sigma', 1)
    mu = pm.Deterministic('mu', a + bA*df2['A'])
    happiness = pm.Normal('happiness', mu=mu, sd=sigma, observed=df2['happiness'])
    m6_10_trace = pm.sample(1000, tune=1000)

# show model summary
varnames = ['~mu']
az.summary(m6_10_trace, varnames, kind="stats", round_to=2, credible_interval=0.89)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, bA, a]
Sampling 4 chains, 0 divergences: 100%|██████████| 8000/8000 [00:02<00:00, 2866.99draws/s]


,mean,sd,hpd_5.5%,hpd_94.5%
a,0.00,0.08,-0.12,0.13
bA,-0.00,0.13,-0.22,0.21
sigma,1.22,0.03,1.17,1.26


In [14]:
# point wise values
m6_9_loo = pm.stats.loo(m6_9_trace, pointwise=False, reff=None, scale='deviance')
m6_10_loo = pm.stats.loo(m6_10_trace, pointwise=False, reff=None, scale='deviance')

m6_9_waic = pm.stats.waic(m6_9_trace, pointwise=False, scale='deviance')
m6_10_waic = pm.stats.waic(m6_10_trace, pointwise=False, scale='deviance')

Model m6.9 contains both marriage status and age. Model m6.10 contains only age. Model m6.9 produces a confounded inference about the relationship between age and happiness, due to opening a collider path. 

In [15]:
pm.compare({'m6_9': m6_9_trace, 'm6_10': m6_10_trace}, ic='WAIC', scale='deviance')

,rank,waic,p_waic,d_waic,weight,se,dse,warning,waic_scale
m6_9,0,2724.43,3.66643,0,1,37.0843,0,False,deviance
m6_10,1,3102.02,2.39316,377.588,1.78817e-59,27.883,35.534,False,deviance


The model that produces the invalid inference, m6.9, is expected to predict much better. And it would. This is because the collider path does convey actual association. We simply end up mistaken about the causal inference. We should not use WAIC (or LOO) to choose among models, unless we have some clear sense of the causal model. These criteria will happily favor confounded models.

### 3. Reconsider the urban fox analysis from last week’s homework. Use WAIC or LOO based model comparison on five different models, each using weight as the outcome, and containing these sets of predictor variables:
1. avgfood + groupsize + area 
2. avgfood + groupsize
3. groupsize + area
4. avgfood
5. area

### Can you explain the relative differences in WAIC scores, using the fox DAG from last week’s homework? Be sure to pay attention to the standard error of the score differences (dSE).

In [16]:
# load data
df = pd.read_csv("Data/foxes.csv", sep=";", header=0)
df.head()

,group,avgfood,groupsize,area,weight
0,1,0.37,2,1.09,5.02
1,1,0.37,2,1.09,2.84
2,2,0.53,2,2.05,5.33
3,2,0.53,2,2.05,6.07
4,3,0.49,2,2.12,5.85


In [17]:
def standardize(series):
    """Standardize a pandas series"""
    std_series = (series - series.mean()) / series.std()
    return std_series

In [18]:
# standardize the variables
df['W'] = standardize(df['weight'])
df['A'] = standardize(df['area'])
df['F'] = standardize(df['avgfood'])
df['G'] = standardize(df['groupsize'])
df.head()

,group,avgfood,groupsize,area,weight,W,A,F,G
0,1,0.37,2,1.09,5.02,0.414135,-2.239596,-1.924829,-1.524089
1,1,0.37,2,1.09,2.84,-1.427046,-2.239596,-1.924829,-1.524089
2,2,0.53,2,2.05,5.33,0.675954,-1.205508,-1.118035,-1.524089
3,2,0.53,2,2.05,6.07,1.300942,-1.205508,-1.118035,-1.524089
4,3,0.49,2,2.12,5.85,1.115135,-1.130106,-1.319734,-1.524089


In [19]:
# define model 1: weight ~ avgfood + groupsize + area
with pm.Model() as m_foxes_1: 
    a = pm.Normal('a', mu=0, sigma=0.25)
    bA = pm.Normal('bA', mu=0, sd=0.5)
    bF = pm.Normal('bF', mu=0, sd=0.5)
    bG = pm.Normal('bG', mu=0, sd=0.5)
    sigma = pm.Exponential('sigma', 1)
    mu = pm.Deterministic('mu', a+bF*df['F']+bG*df['G']+bA*df['A'])
    weight = pm.Normal('weight', mu=mu, sd=sigma, observed=df['W'])
    m_foxes_1 = pm.sample(1000, tune=1000)
    
# define model 2: weight ~ avgfood + groupsize 
with pm.Model() as m_foxes_2: 
    a = pm.Normal('a', mu=0, sigma=0.25)
    bF = pm.Normal('bF', mu=0, sd=0.5)
    bG = pm.Normal('bG', mu=0, sd=0.5)
    sigma = pm.Exponential('sigma', 1)
    mu = pm.Deterministic('mu', a+bF*df['F']+bG*df['G'])
    weight = pm.Normal('weight', mu=mu, sd=sigma, observed=df['W'])
    m_foxes_2 = pm.sample(1000, tune=1000)
    
# define model 3: weight ~ groupsize + area
with pm.Model() as m_foxes_3: 
    a = pm.Normal('a', mu=0, sigma=0.25)
    bA = pm.Normal('bA', mu=0, sd=0.5)
    bG = pm.Normal('bG', mu=0, sd=0.5)
    sigma = pm.Exponential('sigma', 1)
    mu = pm.Deterministic('mu', a+bG*df['G']+bA*df['A'])
    weight = pm.Normal('weight', mu=mu, sd=sigma, observed=df['W'])
    m_foxes_3 = pm.sample(1000, tune=1000)
    
# define model 4: weight ~ avgfood  
with pm.Model() as m_foxes_4: 
    a = pm.Normal('a', mu=0, sigma=0.25)
    bF = pm.Normal('bF', mu=0, sd=0.5)
    sigma = pm.Exponential('sigma', 1)
    mu = pm.Deterministic('mu', a+bF*df['F'])
    weight = pm.Normal('weight', mu=mu, sd=sigma, observed=df['W'])
    m_foxes_4 = pm.sample(1000, tune=1000)
    
# define model 5: weight ~ area
with pm.Model() as m_foxes_5: 
    a = pm.Normal('a', mu=0, sigma=0.25)
    bA = pm.Normal('bA', mu=0, sd=0.5)
    sigma = pm.Exponential('sigma', 1)
    mu = pm.Deterministic('mu', a+bA*df['A'])
    weight = pm.Normal('weight', mu=mu, sd=sigma, observed=df['W'])
    m_foxes_5 = pm.sample(1000, tune=1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, bG, bF, bA, a]
Sampling 4 chains, 0 divergences: 100%|██████████| 8000/8000 [00:03<00:00, 2067.79draws/s]
The acceptance probability does not match the target. It is 0.8792027944363429, but should be close to 0.8. Try to increase the number of tuning steps.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, bG, bF, a]
Sampling 4 chains, 0 divergences: 100%|██████████| 8000/8000 [00:02<00:00, 2686.11draws/s]
The acceptance probability does not match the target. It is 0.8820526973954916, but should be close to 0.8. Try to increase the number of tuning steps.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, bG, bA, a]
Sampling 4 chains, 0 divergences: 100%|██████████| 8000/8000 [00:02<00:00, 311

In [20]:
pm.compare({'1 avgfood + groupsize + area': m_foxes_1, 
            '2 avgfood + groupsize': m_foxes_2,
            '3 groupsize + area': m_foxes_3,
            '4 avgfood': m_foxes_4,
            '5 area': m_foxes_5}, ic='LOO', scale='deviance')

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
1 avgfood + groupsize + area,0,323.091,4.58041,0,0.308131,15.1865,0,False,deviance
2 avgfood + groupsize,1,323.694,3.5376,0.603416,0.34431,14.9885,3.36144,False,deviance
3 groupsize + area,2,323.986,3.65272,0.895434,0.29789,14.7683,2.71405,False,deviance
4 avgfood,3,333.49,2.38616,10.3996,0.0258042,12.8784,6.73483,False,deviance
5 area,4,333.851,2.63762,10.7603,0.0238657,12.8899,6.78881,False,deviance


Notice that the top three models are m1, m3, and m2. They have very similar WAIC values. The differences are small and smaller in all cases than the standard error of the difference. WAIC sees these models are tied. This makes sense, given the DAG, because as long as a model has groupsize in it, we can include either avgfood or area or both and get the same inferences. Another way to think of this is that the influence of good, adjusting for group size, is (according to the DAG) the same as the influence of area, adjusting for group size, because the influence of area is routed entirely through food and group size. There are no backdoor paths.

What about the other two models, m4 and m5? These models are tied with one another, and both omit group size. Again, the influence of area passes entirely through food. So including only food or only area should produce the same inference --- the total causal influence of area (or food) is just about zero. That’s indeed what the posterior distributions suggest:

In [21]:
# show model summary
varnames = ['~mu']
az.summary(m_foxes_4, varnames, kind="stats", round_to=2, credible_interval=0.89)

,mean,sd,hpd_5.5%,hpd_94.5%
a,0.00,0.09,-0.15,0.14
bF,-0.02,0.09,-0.18,0.12
sigma,1.01,0.07,0.91,1.12


In [22]:
# show model summary
varnames = ['~mu']
az.summary(m_foxes_5, varnames, kind="stats", round_to=2, credible_interval=0.89)

,mean,sd,hpd_5.5%,hpd_94.5%
a,-0.00,0.09,-0.14,0.15
bA,0.02,0.09,-0.12,0.17
sigma,1.01,0.07,0.90,1.12
